Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

In [1]:
import datetime
import findspark
import os

In [2]:
ts=datetime.datetime.today()
home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
spark_home = f"{home_dir}/dist/spark-3.4.0-bin-hadoop3"
cores_per_exec = 1
spark_master = f"local[{cores_per_exec}]"
findspark.init(spark_home=spark_home)

In [3]:
import pyspark
spark_rapids_conf = pyspark.SparkConf(loadDefaults=False)
spark = pyspark.sql.SparkSession.builder\
    .config(conf=spark_rapids_conf)\
    .appName('RAPIDS PySpark Notebook')\
    .master(spark_master)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 20:14:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set('spark.sql.session.timeZone', 'UTC')
spark.conf.set('spark.sql.timestampType', 'TIMESTAMP_LTZ')
conf1 = spark.conf
spark.createDataFrame([(ts,),], 'ts timestamp').createOrReplaceTempView('df1')
spark.sql("SELECT * FROM df1").show(truncate=False)

+--------------------------+
|ts                        |
+--------------------------+
|2023-06-06 03:14:30.324271|
+--------------------------+



In [5]:
spark.conf.set('spark.sql.timestampType', 'TIMESTAMP_NTZ')
conf2 = spark.conf
spark.createDataFrame([(ts,),], 'ts timestamp').createOrReplaceTempView('df2')
spark.sql("SELECT * FROM df2").show(truncate=False)

+--------------------------+
|ts                        |
+--------------------------+
|2023-06-05 20:14:30.324271|
+--------------------------+



In [6]:
conf1 == conf2

True

In [7]:
spark.sql('SELECT * FROM df1 JOIN df2').show(truncate=False)

+--------------------------+--------------------------+
|ts                        |ts                        |
+--------------------------+--------------------------+
|2023-06-06 03:14:30.324271|2023-06-05 20:14:30.324271|
+--------------------------+--------------------------+



In [8]:
spark.sql('SELECT * FROM df1 JOIN df2 ON df1.ts=df2.ts').show(truncate=False)

+---+---+
|ts |ts |
+---+---+
+---+---+

